In [8]:
from web3 import Web3

from web3.auto import w3
from web3.middleware.geth_poa import geth_poa_middleware

from eth_abi.abi import decode

from dotenv import load_dotenv

import os

load_dotenv(override=True)


True

In [2]:
web3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/db0355f9beed4af58889a043d591db91"))

In [3]:
tx_hash = "0xc3ce97ac430ac0cf5a56bbc1738619a9df9901320468629899b46a26b2118223"
tx = web3.eth.get_transaction(tx_hash)


In [4]:
tx

AttributeDict({'accessList': [],
 'blockHash': HexBytes('0x3456aa33f6569f988cbbe0d310655be3790e7bde606057c311440cdac3d38ffa'),
 'blockNumber': 17642851,
 'chainId': 1,
 'from': '0x1c9D31413166Cb137769385530c7eBE386723e58',
 'gas': 277648,
 'gasPrice': 38854537533,
 'hash': HexBytes('0xc3ce97ac430ac0cf5a56bbc1738619a9df9901320468629899b46a26b2118223'),
 'input': '0x24856bc300000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000001200000000000000000000000001c9d31413166cb137769385530c7ebe386723e5800000000000000000000000000000000000000000f46251810dc0fc2b696038a0000000000000000000000000000000000000000000000000

In [9]:
# Apply the necessary middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Get trace
trace = w3.manager.request_blocking(f"debug_traceTransaction", [tx_hash, {}])

# Extract calls to swap function
swap_calls = []
for action in trace['structLogs']:
    if action['op'] == 'CALL':
        input_data = action['stack'][-1]
        method_id = w3.keccak(text='swap(address,bool,int256,uint160,bytes)')[:10].hex()
        if input_data[:10] == method_id:
            sqrt_price_limit_x96_hex = input_data[138:202]
            sqrt_price_limit_x96 = int(sqrt_price_limit_x96_hex, 16)
            swap_calls.append(sqrt_price_limit_x96)

print(swap_calls)

CannotHandleRequest: Could not discover provider while making request: method:debug_traceTransaction
params:['0xc3ce97ac430ac0cf5a56bbc1738619a9df9901320468629899b46a26b2118223', {}]


In [31]:
# Retrieve the transaction data
input_data = tx['input']

# Function selector for 'exactInputSingle' function in Uniswap V3's contract
function_selector = w3.to_hex(w3.sha3(text="exactInputSingle((address,address,uint24,address,uint256,uint256,uint256,uint160))")[:4])

# Check if the transaction's input data starts with the correct function selector
if not input_data.startswith(function_selector):
    raise ValueError("Transaction didn't call the 'exactInputSingle' function")

# Remove the function selector from the input data
input_data = input_data[len(function_selector):]

# Decode the remaining input data
params = decode(
    ['address', 'address', 'uint24', 'address', 'uint256', 'uint256', 'uint256', 'uint160'],
    bytes.fromhex(input_data)
)

# Extract the amountOutMinimum and sqrtPriceLimitX96 parameters
amountOutMinimum = params[6]
sqrtPriceLimitX96 = params[7]

AttributeError: 'Web3' object has no attribute 'sha3'

In [23]:
print(f"Amount in: {amount_in}")
print(f"Token in: {token_in}")
print(f"Token out: {token_out}")
print(f"Fee: {fee}")
print(f"Deadline: {deadline}")
print(f"Amount out minimum: {amount_out_minimum}")
print(f"Square root price limit: {sqrt_price_limit}")


NameError: name 'amount_in' is not defined